In [2]:
from gurobipy import Model, GRB, quicksum
import pandas as pd
from code_map import meters, markets
import numpy as np
import matplotlib.pyplot as plt
import calendar 
from datetime import datetime
import pytz
import openpyxl
from itertools import combinations, product


In [3]:
hour = pd.Timestamp(year= 2023, month= 6, day = 26, hour = 16, tz = "Europe/Oslo")
#market_names = [market.name for market in market_list]
power_meter_dict = meters.power_meters

# Define the sets
L = list(power_meter_dict.values())  # List of PowerMeter objects
M = markets.all_market_list  # List of ReserveMarket objects

In [4]:
len(L)

1459

In [5]:
for i in [asset.flex_volume["value"].loc[asset.flex_volume["Time(Local)"] == hour].values[0] for asset in L]:
    # check if there is any nan values
    if np.isnan(i):
        print("There is a nan value in the flex volume")
        print(i)

In [6]:
sum([asset.flex_volume["value"].loc[asset.flex_volume["Time(Local)"] == hour].values[0] for asset in L])

10.766037500000033

In [7]:
[market.min_volume for market in M]

[5, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10]

In [8]:
print("market directions : , ")
print([market.direction for market in M])
print("asset directions : , ")
print([asset.direction for asset in L])

market directions : , 
['up', 'up', 'up', 'both', 'up', 'both', 'up', 'down', 'up', 'down', 'up', 'down', 'up', 'down']
asset directions : , 
['both', 'both', 'both', 'both', 'both', 'down', 'both', 'both', 'both', 'both', 'both', 'both', 'both', 'both', 'both', 'up', 'both', 'both', 'both', 'both', 'both', 'both', 'both', 'both', 'both', 'up', 'both', 'both', 'both', 'both', 'down', 'both', 'both', 'both', 'both', 'both', 'both', 'both', 'both', 'both', 'down', 'both', 'both', 'both', 'both', 'down', 'both', 'both', 'both', 'both', 'both', 'both', 'both', 'both', 'both', 'up', 'both', 'both', 'both', 'both', 'both', 'both', 'both', 'both', 'both', 'up', 'both', 'both', 'both', 'both', 'down', 'both', 'both', 'both', 'both', 'down', 'both', 'both', 'both', 'both', 'down', 'both', 'both', 'both', 'both', 'down', 'both', 'both', 'both', 'both', 'both', 'both', 'both', 'both', 'both', 'down', 'both', 'both', 'both', 'both', 'both', 'both', 'both', 'both', 'both', 'down', 'both', 'both', '

In [9]:
[market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1] for market in M]

[0.0,
 12.750000000000002,
 6.0,
 47.14,
 0.0,
 33.74,
 20.0,
 20.0,
 64.55,
 45.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [10]:
[market.sleep_time for market in M]

[15, 15, 60, 60, 60, 60, 60, 60, 0, 0, 60, 60, 480, 480]

In [11]:
[market.response_time for market in M]

[1.3, 1.3, 30, 30, 30, 30, 300, 300, 300, 300, 300, 300, 300, 300]

In [13]:
# Create a new model
test_model = Model("AssetToMarket")

# Create decision variables
x = {}
for i, asset in enumerate(L):
    for j, market in enumerate(M):
        x[i, j] = test_model.addVar(lb = 0, ub = 1, vtype=GRB.BINARY, name=f"x_{i}_{j}")
        
        # If asset direction is "up"
        if asset.direction == "up" and market.direction not in ["up", "both"]:
            test_model.addConstr(x[ i, j] == 0, f"direction_constraint_hour_{hour}_asset_{asset}_market_{market}")

        # If asset direction is "down"
        elif asset.direction == "down" and market.direction not in ["down", "both"]:
            test_model.addConstr(x[ i, j] == 0, f"direction_constraint_hour_{hour}_asset_{asset}_market_{market}")
y = {}

for j, market in enumerate(M):
    y[j] = test_model.addVar(lb = 0, ub = 1, vtype=GRB.BINARY, name=f"y_{j}")


# Set objective
test_model.setObjective(
    sum(x[i, j] * asset.flex_volume["value"].loc[asset.flex_volume["Time(Local)"] == hour].values[0] * market.price_data.loc[(market.price_data["Time(Local)"] == hour)].values[0][1] for i, asset in enumerate(L) for j, market in enumerate(M)),
    GRB.MAXIMIZE
)

# Add constraints
for i, asset in enumerate(L):
    test_model.addConstr(sum(x[i, j] for j in range(len(M))) <= 1, f"single_market_for_asset_nr.{i}")

for j, market in enumerate(M):
    
    #print("min_volume : ", market.min_volume)
    #print("y[j] : ", y[j])
    #print("sum of x ", sum(x[i, j] for i, asset in enumerate(L)))
    #print("flex volume : ", [asset.flex_volume["value"].loc[(asset.flex_volume["Time(Local)"] == hour)].values[0] for i, asset in enumerate(L)])
    #
    test_model.addConstr((sum(x[i, j] for i in range(len(L)))) <= len(L) * y[j], f"asset_connection_for_market_{market.name}")

    test_model.addConstr(sum(x[i, j] * asset.flex_volume["value"].loc[asset.flex_volume["Time(Local)"] == hour].values[0] for i, asset in enumerate(L)) >= market.min_volume * y[j], f"min_volume_for_market_{market.name}")
    
    #test_model.addConstr(np.mean([x[i, j] * asset.sleep_time for i, asset in enumerate(L)]) <= market.sleep_time, f"avg_sleep_time_for_market_{j}")
    test_model.addConstr(np.mean([x[i, j] * asset.response_time for i, asset in enumerate(L)]) <= market.response_time, f"avg_response_time_for_market_{market.name}")

    test_model.addConstr(sum(x[i, j] * asset.flex_volume["value"].loc[asset.flex_volume["Time(Local)"] == hour].values[0] for i, asset in enumerate(L)) <=  market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1]  * y[j], f"max_volume_for_market_{market.name}")

# Solve the model
test_model.optimize()




Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2575 rows, 20440 columns and 101208 nonzeros
Model fingerprint: 0x6c6c6be7
Variable types: 0 continuous, 20440 integer (20440 binary)
Coefficient statistics:
  Matrix range     [3e-06, 1e+03]
  Objective range  [2e-05, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 1167 rows and 12564 columns
Presolve time: 0.14s
Presolved: 1408 rows, 7876 columns, 35681 nonzeros
Variable types: 0 continuous, 7876 integer (7876 binary)

Root relaxation: objective 4.966676e+02, 1448 iterations, 0.04 seconds (0.07 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  496.66762    0  114   -0.00000  496.66762 

In [ ]:
data = []

for i, asset in enumerate(L):
    for j, market in enumerate(M):
        if x[i, j].X > 0.5:
            data.append([hour, asset.meter_id, market.name])

df = pd.DataFrame(data, columns=["Hour", "Asset Meter ID", "Market"])
#print(df)
hour_df = df.loc[(df["Hour"] == hour)]
# get the number of assets in each market
market_count = hour_df.groupby(["Market", "Hour"]).agg("count").reset_index().rename(columns={"Asset Meter ID": "Asset Count"})
display(market_count)
    

In [ ]:
# Retrieve and print solution (optional)
if test_model.status == GRB.Status.OPTIMAL:
    for i, asset in enumerate(L):
        for j, market in enumerate(M):
            if x[i, j].X > 0.5:
                print(f"Asset with meter id: {asset.meter_id} is bid in to {market.name}")
else:
    print("Optimization was not successful!")
    
if test_model.status == GRB.Status.INFEASIBLE:
    test_model.computeIIS()